In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD import CRUD

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
db = CRUD(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.readAll({}))
df = df.iloc[:,1:]
# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)


## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

image_filename = 'Grazioso Salvare Logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    
    #logo image, title, and creator information
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Center(html.B(html.H1('Grasioso Salvare Austin Animal Center'))),
    html.Center(html.H3('Created by: Zac McBride')),
    
    html.Hr(),
    # row of buttons for filters
    html.Div(
        dcc.Dropdown(
            id='rescue-filter',
            options=[
                {'label':'Water Rescue', 'value':'WR'},
                {'label':'Mountain and Wilderness Rescue', 'value':'MWR'},
                {'label':'Disater Rescue and Individual Tracking', 'value':'DR/IR'},
                {'label': 'Reset', 'value':'RESET'}
            ],
            value='RESET',
            style={"width": "50%"}
        )
    ),

    # Add table to layout           
                
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
    
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),
              [dash.dependencies.Input('rescue-filter', 'value')])
def update_dashboard(value):
    
    # For Water Rescue
    if (value =='WR'):
        df = pd.DataFrame(list(db.readAll(
            { 
            "breed": {
                "$in": [
                    "Chesapeake Bay Retriever",
                    "Labrador Retriever Mix",
                    "Newfoundland"
                ]
            },
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {
                "$gte": 26.0
            },
            "$and": [
                {
                    "age_upon_outcome_in_weeks": {
                        "$lte": 156.0
                    }
                }
            ]
        }
        )))
        
    # For Mountain and Wildlife Rescue
    elif (value == 'MWR'):
        df = pd.DataFrame(list(db.readAll(
            { 
            "breed": {
                "$in": [
                    "German Shepherd",
                    "Alaskan Malamute",
                    "Old English Sheepdog",
                    "Siberian Husky",
                    "Rottweiler"
                ]
            },
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {
                "$gte": 26.0
            },
            "$and": [
                {
                    "age_upon_outcome_in_weeks": {
                        "$lte": 156.0
                    }
                }
            ]
        }
        )))
        
        # For Disaster Rescue and Individual Tracking
    elif (value == 'DR/IR'):
        df = pd.DataFrame(list(db.readAll(
            { 
            "breed": {
                "$in": [
                    "Doberman Pinscher",
                    "German Shepherd",
                    "Golden Retriever",
                    "Bloodhound",
                    "Rottweiler"
                ]
            },
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {
                "$gte": 20.0
            },
            "$and": [
                {
                    "age_upon_outcome_in_weeks": {
                        "$lte": 300.0
                    }
                }
            ]
        }
        )))
    
    # For nothing selected
    else:
        df = pd.DataFrame.from_records(db.readAll({}))
        
    return df.to_dict('records')

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    # imports the currently displayed data
    dff = pd.DataFrame.from_dict(viewData)
    #creates the values needed for the names (breed) and values (recurring counts)
    names = dff['breed'].value_counts().keys().tolist()
    values = dff['breed'].value_counts().tolist()
    #creates a pie chart based on the data above
    return [
        dcc.Graph(            
            figure = px.pie(
                data_frame=dff, 
                values = values, 
                names = names, 
                title = "Amount of Each Breed",
                color_discrete_sequence=px.colors.sequential.RdBu,
                width=800, 
                height=500  
            )
        )
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', 'selected_rows')])
def update_map(viewData, selectedRows):   
    
    # Get selected row, or default to first row if no selection
    if not selectedRows:
        selected_row = 0
    else:   
        selected_row = selectedRows[0]
        
                           
    dff = pd.DataFrame.from_dict(viewData)   
    
    # Get coordinate position data from selected row
    posX = dff.iloc[selected_row, 13]
    posY = dff.iloc[selected_row, 14]
    
    # Get name from selected row
    # If name is empty replace with "No name"
    name = dff.iloc[selected_row,9] 
    if name == "":
        name = "No name"
        
    # Return a map using data pulled from selected row
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[posX,posY], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[posX,posY], children=[
                dl.Tooltip(dff.iloc[selected_row,4]),
                dl.Popup([
                    html.P("Animal Name"),
                    html.H1(name)
                ])
            ])
        ])
    ]

app.run_server(debug=True)


Successfully loaded database
Dash app running on http://127.0.0.1:18966/
